# QOSF Mentorship Assessment Task 
## Submission by Kartikeya Rambhatla

## Task 3 

Learning by doing: the best way to understand the basics of quantum computation is to implement a quantum circuit simulator. This task is suitable both for people from computer sciences who want to learn about quantum computing, and for people from math/physics who want to exercise coding. It is expected that simulator can perform following:

●	initialize state <br>
●	read program, and for each gate: <br>
●	calculate matrix operator <br>
●	apply operator (modify state) <br>
●	perform multi-shot measurement of all qubits using weighted random technique

## Bonus requirements

●	Parametric gates <br>
●	Allow running variational quantum algorithms <br>


## Solution

My goal is to create a quantum circuit simulator which can handle the creation of states, application of gates, calculation of matrix operators of the corresponding gates, and perform multi-shot measurement on the circuit.

In [1]:
import numpy as np
import scipy as sp
import scipy.linalg
import sympy as smp
from sympy import Symbol

## Introduction

The basic |0> and |1> qubit statevectors can be reprsented as the two orthonormal basis vectors (1, 0) and (0, 1) using numpy in the correct dimensions. 

Simiarly it is useful to have projectors of these basis vectors if we want to perform any kind of tensor operations on these qubits. The projectors can be written as: P0 = |0><0| and P1 = |1><1| for the |0> and |1> basis states respectively. 

In [2]:
# Qubit Basis vectors

q_0 = np.array([[1], [0]])
q_1 = np.array([[0], [1]])

# Identity matrix and the projection vectors
Identity= np.eye(2)
P0 = np.dot(q_0, q_0.T)
P1 = np.dot(q_1, q_1.T)

# Defining some basic gate operations

Hadamard = 1/np.sqrt(2) * np.array([[1, 1],
                            [1, -1]])

NOT = np.array([[0, 1],
              [1, 0]])

Y = np.array([[0, -1j],
              [-1j, 0]])

Z = np.array([[1, 0],
              [0, -1]])

S = np.array([[1, 0],
              [0, 1j]])

def U3(theta, phi, lam):
    ''' A function that returns U3 matrix operation based on the values of theta, phi and lam(lambda). The arguments are returned as a sympy expression in order to handle strings as inputs and extend support for parametric gates.'''

    return np.array([[smp.cos(theta/2), -smp.exp(1j*lam) * smp.sin(theta/2)],
            [smp.exp(1j*phi) * smp.sin(theta/2), smp.exp(1j*lam + 1j*phi) * smp.cos(theta/2)]])

## Multiple qubits and the Tensor product 

In order to generalize matrix operations to multiple qubits, we can represent the statevectors of multiple qubits as tensor products of the individual qubit states. So the state |101> = |1> ⊗ |0> ⊗ |1>. Since the single qubit is a 2-dimensional vector, so the tensor product of $N$ qubit states creates a statevector of the dimension $2^N$.

The tensor product operation for two matrices (let's say M1 and M2) can be performed in numpy using the Kronecker product function np.kron(M1, M2) which will return $M1 ⊗ M2$. In order to extend this to multiple product operation, we can just use a loop.

In [3]:
# The multi-tensor product function
def multikron(M_list):
    temp = np.array([[1]])
    for matrix in M_list:
        temp = np.kron(temp, matrix)
    return temp

# A function that convers an integer to its binary form of a given size. Useful for generating state names in the measurement function for returning the corresponding counts. 
def int_to_bin(num, size):
    return format(num, "b").zfill(size)

## Gates 

In order to perform operations on the qubits, we need to have gates as matrix operators of the proper dimension. We can create a basic $Gate$ class that can be used as a parent for all the other gate types. The gates are initialized using the size of the circuit in order to calculate the proper dimensions, the control/target qubits and the corresponding unitary to create the necessary matrix operator.

The $UGate$ class creates a unitary matrix that acts on a single qubit. This can be used to create all the basic gate operations on the circuit. It is a simple tensor product of the given unitary on the target qubit and the identity for the rest of the qubits. For example: If the unitary $U$ is applied on the $2^{nd}$ qubit of a 3-qubit circuit, then the corresponding matrix operator will be $I ⊗ U ⊗ I$.

The $CGate$ class is a little bit more complicated and performs a controlled-$U$ operation for a given unitary $U$. This class needs a $UGate$ object as an input in order to create a $CU$ operation. For example: If a unitary $U$ is to be applied on the target qubit 3 with the control qubit 1, for a 3 qubit circuit then the corresponsing matrix operation will be:$$
CU_{3, 1} = I ⊗ I ⊗ |0><0| + U ⊗ I ⊗ |1><1|$$

In [4]:
class Gates:
    def __init__(self, n):
        self.n = n
        self.matrix_op = Identity
        self.label = None
        self.parametric = False

    def get_operator(self):
        return self.matrix_op 

class UGate(Gates):

    def __init__(self,n, target, unitary):
        super().__init__(n)

        self.target = target
        self.unitary = unitary
        self.label = 'U'

        result = np.array([[1.0]])

        for i in range(self.n):
            if i == target:
                op = unitary
            else:
                op = Identity

            result = np.kron(result, op)

        self.matrix_op = result

class CGate(Gates):
    def __init__(self, n, controls, targetGate):
        super().__init__(n)
        self.label = 'Controlled ' + targetGate.label

        L1 = []
        L2 = []
        for i in range(n):
            if i in controls:
                L1 += [P0]
                L2 += [P1]
            else:
                L1 += [Identity]
                L2 += [Identity]   

        L2[targetGate.target] = targetGate.unitary
            
        
        self.matrix_op = multikron(L1) + multikron(L2) #+ targetGate.get_operator()

## Quantum Circuit

In order to handle quantum circuits, we create a $Quantum\_Circuit$ class which contains the statevector, methods for basic gates, and the circuit operation as a list in the sequential order. New gates can be added by extending the class and creating custom gates using the $UGate$ and the $CGate$ classes.

The class also allows for the usage of $\textbf{parametric gates}$ as mentioned in the bonus requirements which is handled using the $SymPy$ module. If a string is used as an input, the string is converted into a $SymPy$ symbol and the expression is stored in a symbolic way. The parameters can later be binded using the $bind_parameters$ method and passing the parameter values as a dictionary. This is evaulated using $SymPy's$ evalf() function and converted into a numpy array for later usage.

The $measurement$ method performs multi-shot measurements using weighted probabilities generated from the square of the absolute value of the statevectors. Since it uses the $numpy$ weighted random function, it can perform high speed measurements for a large number of shots.

### NOTE: The qubits are ordered from left to right. So the state |100> corresponds to the qubit 0 being in |1> state, and qubit 1 and qubit 2 in the |0> state. Also, since the statevector size grows exponentially ($2^N$) with the size of the circuit, large circuits can crash the program.

In [5]:
class Quantum_Circuit:

    def __init__(self, n):
        self.n = n
        self.statevector = multikron(([q_0]*n))
        self.circuit = []

    def u3(self, theta, phi, lam, q1):
        parametric = False
        if isinstance(theta, str):
            theta = Symbol(theta)
            parametric = True
        if isinstance(phi, str):
            phi = Symbol(phi)
            parametric = True
        if isinstance(lam, str):
            lam = Symbol(lam)
            parametric = True

        u = smp.Matrix(U3(theta, phi, lam))
        operator = UGate(self.n, q1, u)
        operator.parametric = parametric
        self.qapply(operator)

    def h(self, q1):
        operator = UGate(self.n, q1, Hadamard)
        self.qapply(operator)

    def x(self, q1):
        operator = UGate(self.n, q1, NOT)
        self.qapply(operator)

    def y(self, q1):
        operator = UGate(self.n, q1, Y)
        self.qapply(operator)

    def z(self, q1):
        operator = UGate(self.n, q1, Z)
        self.qapply(operator)

    def s(self, q1):
        operator = UGate(self.n, q1, S)
        self.qapply(operator)

    def cx(self, q1, q2):
        operatorX = UGate(self.n, q2, NOT)
        operatorC = CGate(self.n, [q1], operatorX)
        self.qapply(operatorC)        
    
    def cz(self, q1, q2):
        operatorZ = UGate(self.n, q2, Z)
        operatorC = CGate(self.n, [q1], operatorZ)
        self.qapply(operatorC)

    def qapply(self, operator):
        self.circuit.append(operator)

    def measurement(self, shots):
        self.statevector = self.get_statevector()
        
        state_list = []
        for i in range(len(self.statevector)):
            state_list.append(int_to_bin(i, self.n))
        
        prob = np.squeeze(np.square(np.abs(self.statevector)))
        outcomes = np.random.choice(state_list, shots, p = prob)

        outstate, counts = np.unique(outcomes, return_counts=True)
        result = dict(zip(outstate, counts))
        return result

    def get_statevector(self):
        sv = np.array(self.statevector)
        for operator in self.circuit:
            op_matrix = operator.get_operator()
            sv = np.dot(op_matrix, sv)
        sv = np.squeeze(np.fromiter(sv, dtype=complex))
        return sv

    def bind_parameters(self, params_dict):
        for operator in self.circuit:
            if operator.parametric:
                x = np.array(smp.Matrix(operator.get_operator()).evalf(subs=params_dict))
                operator.matrix_op = x
                operator.parametric = False
        return self


## Testing the program

1. A simple 2-qubit circuit for creating the bell state is implemented. 
2. The parametric U3 gate is used for using a string as an input and later a value is passed into it

In [6]:
q = Quantum_Circuit(2)
q.h(0)
q.cx(0, 1)
print(list(q.get_statevector()))
q.measurement(100000)

[(0.7071067811865475+0j), 0j, 0j, (0.7071067811865475+0j)]


{'00': 50111, '11': 49889}

In [7]:
q = Quantum_Circuit(2)
q.u3('theta', np.pi/2, np.pi/2, 0)
q.bind_parameters({'theta': np.pi/2})
print(list(q.get_statevector()))
q.measurement(100000)

[(0.7071067811865476+0j), 0j, (4.3297802811774664e-17+0.7071067811865475j), 0j]


{'00': 49947, '10': 50053}

## The $Quantum\_Circuit$ class and the $UGate$ and $CGate$ classes have been generalized in order to add extended features to the program. 